<a href="https://colab.research.google.com/github/sbooeshaghi/BMGP_2020/blob/main/analysis/notebooks/10xFB/10xFB_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!date

Mon Jun 14 22:14:54 UTC 2021


# Install packages and download data

In [2]:
import sys
COLAB = "google.colab" in sys.modules


In [3]:
if COLAB:
    !git clone https://github.com/sbooeshaghi/BMGP_2020.git

Cloning into 'BMGP_2020'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 490 (delta 61), reused 124 (delta 28), pack-reused 296
Receiving objects: 100% (490/490), 425.96 MiB | 31.05 MiB/s, done.
Resolving deltas: 100% (152/152), done.
Checking out files: 100% (130/130), done.


In [4]:
if COLAB:
    !pip install --quiet \
    kb-python==0.26.0 \
    anndata==0.7.6

     |████████████████████████████████| 59.1MB 71kB/s 
     |████████████████████████████████| 133kB 34.3MB/s 
     |████████████████████████████████| 10.3MB 21.5MB/s 
     |████████████████████████████████| 13.2MB 154kB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 112kB 52.3MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 1.2MB 32.2MB/s 


In [ ]:
if COLAB:
    # download cellranger (30sec)
    !wget -O cellranger-6.0.1.tar.gz \
    -q --show-progress --progress=bar:force  \
    "https://cf.10xgenomics.com/releases/cell-exp/cellranger-6.0.1.tar.gz?Expires=1619593023&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZi4xMHhnZW5vbWljcy5jb20vcmVsZWFzZXMvY2VsbC1leHAvY2VsbHJhbmdlci02LjAuMS50YXIuZ3oiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE2MTk1OTMwMjN9fX1dfQ__&Signature=EfUmpmiiOtatL9nfNGsODQnSC2a5GmvKesMiVee~04H2ndGVBNkLGB7uWuU3c1NvgG5R1GMk6MYNaPcTaF100IPncjjnFFot7qawOb1LiZQx7sxXLlDfyEPr4TshlXdwzVkU8DX1yZeq82iHVc3etUHiugVLVttUW7wrfd1do35jVoJzjHacWeFGBS6CWvFpvAhNsT29-YULeJRDbdYe~TYhjsKnl1BVWSQzKUtdNsrsiPgXAnAf4GsNmEDNA~b~Gnwyubs3cGGna4TDXw65IcoduOHcIxOMHlhgX5wIabfcMZx3i9Jaw7TpwFxRHHEpoFftHuEsGHs1lusxB8pKYQ__&Key-Pair-Id=APKAI7S6A5RYOXBWRPDA" \
    2>&1
    
    !tar -xf cellranger-6.0.1.tar.gz

    
    # download cellranger reference (5min)
    !wget \
    -q --show-progress --progress=bar:force  \
    https://cf.10xgenomics.com/supp/cell-exp/refdata-gex-GRCh38-2020-A.tar.gz \
    2>&1
    
    !tar -xf refdata-gex-GRCh38-2020-A.tar.gz


cellranger-6.0.1.ta 100%[===================>]   1.01G  37.7MB/s    in 26s     


In [ ]:
%%bash
source cellranger-6.0.1/sourceme.bash

In [5]:
if COLAB:
    # download the relevant data
    !wget -O 10xFB_5k_pbmc_v3_fastqs.tar.gz \
    -q --show-progress --progress=bar:force  \
    https://caltech.box.com/shared/static/z28rkegt9rdrmjlly9ashv7jqtghztjp.gz \
    2>&1
    !tar -xf 10xFB_5k_pbmc_v3_fastqs.tar.gz

10xFB_5k_pbmc_v3_fa 100%[===================>]   2.60G  18.1MB/s    in 2m 13s  


# Preprocess with kb

In [6]:
%%time
!FB="BMGP_2020/references/10xFB_5k_pbmc_v3/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.idx" && \
F2B="BMGP_2020/references/10xFB_5k_pbmc_v3/kite/f2b.txt" && \
FASTA="BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.fa" && \
kb ref \
-i $INDEX \
-g $F2B \
-f1 $FASTA \
--workflow kite \
$FB

[2021-06-14 22:18:51,938]    INFO Generating mismatch FASTA at BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.fa
[2021-06-14 22:18:51,969]    INFO Creating transcript-to-gene mapping at BMGP_2020/references/10xFB_5k_pbmc_v3/kite/f2b.txt
[2021-06-14 22:18:51,976]    INFO Skipping kallisto index because BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.idx already exists. Use the --overwrite flag to overwrite.
CPU times: user 27 ms, sys: 7.25 ms, total: 34.2 ms
Wall time: 3.85 s


In [7]:
%%bash
FB="BMGP_2020/references/10xFB_5k_pbmc_v3/kite/feature_barcodes.txt" && \
INDEX="BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.idx" && \
F2B="BMGP_2020/references/10xFB_5k_pbmc_v3/kite/f2b.txt" && \
FASTA="BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.fa" && \
OUT="kite_out" && \
FASTQS=$(FASTQDIR="fastqs/" && paste -d" " \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R1) \
<(ls $FASTQDIR | awk -v p=$FASTQDIR '{print p$0}' | grep R2)) && \
kb count \
-i $INDEX \
-g $F2B \
-x 10xv3 \
-o $OUT \
--workflow kite:10xFB \
--h5ad \
--filter bustools \
$FASTQS

[2021-06-14 22:18:53,050]    INFO Using index BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.idx to generate BUS file to kite_out from
[2021-06-14 22:18:53,050]    INFO         fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R1_001.fastq.gz
[2021-06-14 22:18:53,050]    INFO         fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R2_001.fastq.gz
[2021-06-14 22:18:53,050]    INFO         fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R1_001.fastq.gz
[2021-06-14 22:18:53,050]    INFO         fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R2_001.fastq.gz
[2021-06-14 22:23:11,615]    INFO Sorting BUS file kite_out/output.bus to kite_out/tmp/output.s.bus
[2021-06-14 22:23:39,678]    INFO Whitelist not provided
[2021-06-14 22:23:39,678]    INFO Copying pre-packaged 10XV3 whitelist to kite_out
[2021-06-14 22:23:40,590]    INFO Copying 10xv3 feature-to-barcode map to kite_out
[2021-06-14 22:23:42,494]    INFO Projecting BUS file kite_out/tmp/output.s.bus with map kite_out/10xv3_feature_barcode_map.txt
[2021

In [8]:
!cat kite_out/run_info.json

{
	"n_targets": 0,
	"n_bootstraps": 0,
	"n_processed": 39096673,
	"n_pseudoaligned": 37262835,
	"n_unique": 37262835,
	"p_pseudoaligned": 95.3,
	"p_unique": 95.3,
	"kallisto_version": "0.46.2",
	"index_version": 0,
	"start_time": "Mon Jun 14 22:18:53 2021",
	"call": "/usr/local/lib/python3.7/dist-packages/kb_python/bins/linux/kallisto/kallisto bus -i BMGP_2020/references/10xFB_5k_pbmc_v3/kite/features.idx -o kite_out -x 10xv3 -t 8 fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R1_001.fastq.gz fastqs/5k_pbmc_protein_v3_antibody_S2_L001_R2_001.fastq.gz fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R1_001.fastq.gz fastqs/5k_pbmc_protein_v3_antibody_S2_L002_R2_001.fastq.gz"
}


# Preprocess with cellranger

In [ ]:
!printf "fastqs,sample,library_type\n$PWD/fastqs,5k_pbmc_protein_v3_antibody,Antibody Capture" > BMGP_2020/references/10xFB_5k_pbmc_v3/5k_pbmc_protein_v3_libraries.csv

In [ ]:
!LIB="BMGP_2020/references/10xFB_5k_pbmc_v3/5k_pbmc_protein_v3_libraries.csv" && \
FR="BMGP_2020/references/10xFB_5k_pbmc_v3/5k_pbmc_protein_v3_feature_ref.csv" && \
./cellranger-6.0.1/cellranger count \
--libraries $LIB \
--feature-ref $FR \
--id count \
--nosecondary \
--disable-ui \
--expect-cells 5000 \
--transcriptome refdata-gex-GRCh38-2020-A

Martian Runtime - v4.0.4
2021-04-27 13:52:55 [jobmngr] WARNING: configured to use 340GB of local memory, but only 263.3GB is currently available.
Serving UI at http://atlas:42761?auth=UW3XRrq4jVUqusPKpmLnu1rRB_JNV0tF_ZAOk3lxgYg

Running preflight checks (please wait)...
Checking feature definition file...
Checking sample info...
Checking FASTQ folder...
Checking reference...
Checking reference_path (/home/sina/projects/kite/BMGP_2020/analysis/notebooks/refdata-gex-GRCh38-2020-A) on atlas...
Checking optional arguments...
mrc: v4.0.4

mrp: v4.0.4

Anaconda: Python 3.7.6

numpy: 1.19.2

scipy: 1.1.0

pysam: 0.16.0.1

h5py: 2.8.0

pandas: 0.24.2

STAR: 2.7.2a

samtools: samtools 1.10
Using htslib 1.10.2
Copyright (C) 2019 Genome Research Ltd.

2021-04-27 13:54:08 [runtime] (ready)           ID.count.SC_RNA_COUNTER_CS.SC_MULTI_CORE.MULTI_CHEMISTRY_DETECTOR._GEM_WELL_CHEMISTRY_DETECTOR.DETECT_COUNT_CHEMISTRY
2021-04-27 13:54:08 [runtime] (run:local)       ID.count.SC_RNA_COUNTER_CS.SC_MULTI